<a href="https://colab.research.google.com/github/GodJiLee/Dacon/blob/main/%EA%B5%AC%EB%82%B4%EC%8B%9D%EB%8B%B9%20%EC%8B%9D%EC%88%98%EC%9D%B8%EC%9B%90%20%EC%98%88%EC%B8%A1/78_25928_xgboost_feature_Add.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **한글 패치 (matplotlib)**

In [ ]:
# Step 1. Matplotlib 업그레이드
!pip install matplotlib -U

# Step 2. 한글 설치 및 사용 설정
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

# Step 3. 런타임 재시작

Reading package lists... Done
Building dependency tree       
Reading state information... Done
fonts-nanum is already the newest version (20170925-1).
0 upgraded, 0 newly installed, 0 to remove and 40 not upgraded.
/usr/share/fonts: caching, new cache contents: 0 fonts, 1 dirs
/usr/share/fonts/truetype: caching, new cache contents: 0 fonts, 3 dirs
/usr/share/fonts/truetype/humor-sans: caching, new cache contents: 1 fonts, 0 dirs
/usr/share/fonts/truetype/liberation: caching, new cache contents: 16 fonts, 0 dirs
/usr/share/fonts/truetype/nanum: caching, new cache contents: 10 fonts, 0 dirs
/usr/local/share/fonts: caching, new cache contents: 0 fonts, 0 dirs
/root/.local/share/fonts: skipping, no such directory
/root/.fonts: skipping, no such directory
/var/cache/fontconfig: cleaning cache directory
/root/.cache/fontconfig: not cleaning non-existent cache directory
/root/.fontconfig: not cleaning non-existent cache directory
fc-cache: succeeded


In [ ]:
# Step 4. 한글 글꼴 설정
import matplotlib.pyplot as plt

plt.rcParams['font.family']=['NanumGothic', 'sans-serif']
plt.rcParams['axes.unicode_minus'] = False

# **모듈 임포트**

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
# 경고 제어
warnings.filterwarnings(action = 'ignore') # 경고를 출력하지 않음

In [ ]:
from tqdm import tqdm
from sklearn.preprocessing import LabelEncoder # 수치형 자료가 아닌 자료에 대해 모델 학습을 위해 정수형으로 바꿔주는 기능
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from lightgbm import LGBMRegressor # Tree 기반 학습 알고리즘 (Gradient Boosting 프레임워크)

from gensim.models import Word2Vec
from gensim.models import KeyedVectors # 워드 벡터를 저장하고 관리하는 기능, 유사성 조회
from sklearn.manifold import TSNE # PCA 기반 차원 축소의 문제를 해결하기 위한 방법 (티스니)
from sklearn.decomposition import PCA

# 4. 데이터 불러오기

In [ ]:
train = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/dacon_contest1/data/train.csv')
test = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/dacon_contest1/data/test.csv')

# 5. 데이터 정제 (Dtype: object)

1. 조식, 중식, 석식메뉴는 버리기 (drop)
2. 일자, 요일은 라벨 인코딩 (숫자형으로 변환)

## 1. 조식, 중식, 석식메뉴는 버리기 (drop)

In [ ]:
drops = ['조식메뉴', '중식메뉴', '석식메뉴']

train = train.drop(drops, axis=1)
test = test.drop(drops, axis=1)

In [ ]:
train.head()

,일자,요일,본사정원수,본사휴가자수,본사출장자수,본사시간외근무명령서승인건수,현본사소속재택근무자수,중식계,석식계
0,2016-02-01,월,2601,50,150,238,0.0,1039.0,331.0
1,2016-02-02,화,2601,50,173,319,0.0,867.0,560.0
2,2016-02-03,수,2601,56,180,111,0.0,1017.0,573.0
3,2016-02-04,목,2601,104,220,355,0.0,978.0,525.0
4,2016-02-05,금,2601,278,181,34,0.0,925.0,330.0


In [ ]:
test.head()

,일자,요일,본사정원수,본사휴가자수,본사출장자수,본사시간외근무명령서승인건수,현본사소속재택근무자수
0,2021-01-27,수,2983,88,182,5,358.0
1,2021-01-28,목,2983,104,212,409,348.0
2,2021-01-29,금,2983,270,249,0,294.0
3,2021-02-01,월,2924,108,154,538,322.0
4,2021-02-02,화,2924,62,186,455,314.0


## 2. 일자, 요일은 라벨 인코딩 (숫자형으로 변환)

In [ ]:
train['월'] = pd.DatetimeIndex(train['일자']).month
test['월'] = pd.DatetimeIndex(test['일자']).month

train['일'] = pd.DatetimeIndex(train['일자']).day
test['일'] = pd.DatetimeIndex(test['일자']).day

In [ ]:
weekday = {
    '월': 1,
    '화': 2,
    '수': 3,
    '목': 4,
    '금': 5
}

train['요일'] = train['요일'].map(weekday)
test['요일'] = test['요일'].map(weekday)

### **요일(석식) 컬럼 만들기**

In [ ]:
# 요일을 석식 rank에 맞춰 mapping한 요일(석식) 컬럼 만들기
# train 데이터만의 특징일 수 있지 않을까..?
weekday_rank4dinner = {
    1: 1,
    2: 2,
    3: 5,
    4: 3,
    5: 4,
}

train['요일(석식)'] = train['요일'].map(weekday_rank4dinner)
test['요일(석식)'] = test['요일'].map(weekday_rank4dinner)

# 6. 데이터 정제 (아이디어)

* 휴가자, 재택근무자는 중식/석식에 참여하지 않을 것이므로 본사정원수에서 차감한다.
* 즉, **본사정원수 - 휴가자 - 재택근무자**

* 식사참여율 = 식사계 / 식사가능자수

In [ ]:
train['식사가능자수'] = train['본사정원수'] - train['본사휴가자수'] - train['현본사소속재택근무자수']
test['식사가능자수'] = test['본사정원수'] - test['본사휴가자수'] - test['현본사소속재택근무자수']

In [ ]:
train['중식참여율'] = train['중식계'] / train['식사가능자수']
train['석식참여율'] = train['석식계'] / train['식사가능자수']

In [ ]:
# 식사가능자수 float -> int로 변경
train['식사가능자수'] = train['식사가능자수'].astype('int')
test['식사가능자수'] = test['식사가능자수'].astype('int')

In [ ]:
# 현본사소속재택근무자수 float -> int로 변경
train['현본사소속재택근무자수'] = train['현본사소속재택근무자수'].astype('int')
test['현본사소속재택근무자수'] = test['현본사소속재택근무자수'].astype('int')

In [ ]:
# 중식계/석식계 float -> int로 변경
train['중식계'] = train['중식계'].astype('int')
train['석식계'] = train['석식계'].astype('int')

### **공휴일 전후 데이터 추가**
* 테스트 데이터에 설날 연휴, 어린이날 존재
* 연휴 전날 1로 인코딩 나머지 0

In [ ]:
# 새로운 column 생성
train['공휴일전후'] = 0
test['공휴일전후'] = 0

In [ ]:
# 한땀한땀 공휴일 지정..
train['공휴일전후'][17] = 1
train['공휴일전후'][3] = 1
train['공휴일전후'][62] = 1
# train['공휴일전후'][67] = 1
# train['공휴일전후'][82] = 1
train['공휴일전후'][131] = 1
# train['공휴일전후'][130] = 1
train['공휴일전후'][152] = 1
train['공휴일전후'][226] = 1
train['공휴일전후'][221] = 1
train['공휴일전후'][224] = 1
# train['공휴일전후'][244] = 1
train['공휴일전후'][245] = 1
# train['공휴일전후'][267] = 1
train['공휴일전후'][310] = 2
train['공휴일전후'][311] = 1
train['공휴일전후'][309] = 1
train['공휴일전후'][330] = 1
train['공휴일전후'][379] = 1
train['공휴일전후'][467] = 1
# train['공휴일전후'][469] = 1
train['공휴일전후'][470] = 1
train['공휴일전후'][502] = 2
# train['공휴일전후'][501] = 1
# train['공휴일전후'][511] = 1
train['공휴일전후'][565] = 1
train['공휴일전후'][623] = 1
train['공휴일전후'][651] = 1
# train['공휴일전후'][650] = 1
train['공휴일전후'][705] = 1
# train['공휴일전후'][707] = 1
train['공휴일전후'][709] = 1
# train['공휴일전후'][733] = 1
# train['공휴일전후'][748] = 1
# train['공휴일전후'][792] = 1
train['공휴일전후'][815] = 1
train['공휴일전후'][864] = 1
# train['공휴일전후'][863] = 1
train['공휴일전후'][950] = 1
train['공휴일전후'][951] = 1
train['공휴일전후'][953] = 1
train['공휴일전후'][954] = 1
train['공휴일전후'][955] = 1
train['공휴일전후'][971] = 2
# train['공휴일전후'][970] = 1
# train['공휴일전후'][1037] = 1
train['공휴일전후'][1038] = 1
train['공휴일전후'][1099] = 1
train['공휴일전후'][1129] = 1
# train['공휴일전후'][1128] = 1
train['공휴일전후'][1187] = 1
# train['공휴일전후'][1186] = 1

test['공휴일전후'][10] = 2
test['공휴일전후'][20] = 1

**원-핫 인코딩**

In [ ]:
# 변수간 관계성 삭제 (더미화, 가변수화)
train = pd.get_dummies(train, columns = ['공휴일전후'])
test = pd.get_dummies(test, columns = ['공휴일전후'])

In [ ]:
test['공휴일전후_0'][20] = 1
test['공휴일전후_1'][20] = 0

정제한 데이터를 정리하면,

In [ ]:
features_lunch = ['월', '일', '요일', '식사가능자수', '본사휴가자수', '현본사소속재택근무자수', '본사출장자수', '본사시간외근무명령서승인건수', '공휴일전후_0', '공휴일전후_1', '공휴일전후_2']
features_dinner = ['월', '일', '요일(석식)', '식사가능자수', '본사휴가자수', '현본사소속재택근무자수', '본사출장자수', '본사시간외근무명령서승인건수', '공휴일전후_0', '공휴일전후_1', '공휴일전후_2']
labels = ['중식계',	'석식계']

### **모델에 사용할 데이터 생성**

In [ ]:
# lunch

# train
train_x_lunch = train[features_lunch]
train_x_dinner = train[features_dinner]
train_y_lunch = train['중식계']
train_y_dinner = train['석식계']
# test
test_x_lunch = test[features_lunch]
test_x_dinner = test[features_dinner]

In [ ]:
train_x_lunch.head(2)

,월,일,요일,식사가능자수,본사휴가자수,현본사소속재택근무자수,본사출장자수,본사시간외근무명령서승인건수,공휴일전후_0,공휴일전후_1,공휴일전후_2
0,2,1,1,2551,50,0,150,238,1,0,0
1,2,2,2,2551,50,0,173,319,1,0,0


In [ ]:
train_x_dinner.head(2)

,월,일,요일(석식),식사가능자수,본사휴가자수,현본사소속재택근무자수,본사출장자수,본사시간외근무명령서승인건수,공휴일전후_0,공휴일전후_1,공휴일전후_2
0,2,1,1,2551,50,0,150,238,1,0,0
1,2,2,2,2551,50,0,173,319,1,0,0


## **모델링**

In [ ]:
submit = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/dacon_contest1/data/sample_submission.csv')
submit.head()

,일자,중식계,석식계
0,2021-01-27,0,0
1,2021-01-28,0,0
2,2021-01-29,0,0
3,2021-02-01,0,0
4,2021-02-02,0,0


### **1. XGBoost**

In [ ]:
from sklearn.model_selection import GridSearchCV
from xgboost import XGBRegressor

params = {
    'learning_rate': [0.0, 0.1, 0.09, 0.089, 0.08],
    'boosting_type': ['gbtree', 'gblinear', 'dart'],
    
}

lunch_r = XGBRegressor(objective='reg:squarederror')
dinner_r = XGBRegressor(objective='reg:squarederror')

lunch_model = GridSearchCV(lunch_r, params, scoring='neg_mean_absolute_error')
dinner_model = GridSearchCV(dinner_r, params, scoring='neg_mean_absolute_error')

### **중식**

In [ ]:
lunch_model.fit(train_x_lunch, train_y_lunch)
print(lunch_model.best_score_, lunch_model.best_params_)
lunch_model = lunch_model.best_estimator_

-75.53545068764588 {'boosting_type': 'gbtree', 'learning_rate': 0.1}


In [ ]:
y_pred = lunch_model.predict(test_x_lunch)

In [ ]:
submit['중식계'] = y_pred

### **석식**

In [ ]:
dinner_model.fit(train_x_dinner, train_y_dinner)
print(dinner_model.best_score_, dinner_model.best_params_)
dinner_model = dinner_model.best_estimator_

-70.06016379708572 {'boosting_type': 'gbtree', 'learning_rate': 0.08}


In [ ]:
y_pred = dinner_model.predict(test_x_dinner)

In [ ]:
submit['석식계'] = y_pred

### **제출 파일 다운로드**

In [ ]:
submit.to_csv('submit_feature_add_lndn_xgboost.csv', index=False)

from google.colab import files

files.download('submit_feature_add_lndn_xgboost.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

* 1차 시도
  * feature = ['요일', '식사가능자수', '본사출장자수', '본사시간외근무명령서승인건수']
  * score = 94.3871666667

* 2차 시도
  * feature = 1차 시도 + '월'
  * score = 82.6756666667

* 3차 시도
  * weekday rank에 맞춰 매핑
    * 중식,
    * 석식 구분.
  * score = 	81.3001666667

* 4차 시도
  * feature에 '일' 추가
    * both 중식 and 석식
  * score = 80.9271666667

  * 5차 시도
    * XGBoost + GridSearch
    * score = 79.828006

* 6차 시도 (예정)
  * train 데이터에서 석식계가 0인 날 제외 (test에는 석식계 0 없음 확인 필)